# Account - AVR Guest Experience PROD

## Database Summary

In [ ]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Account            = 'DevOps-AVR-Guest-Experience-PROD'
$RDSSummaryColumns  = 'AvailabilityZone', 'DBSnapshotIdentifier', 'SnapshotCreateTime', 'SnapshotType', 'Status'
$Profiles           = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match $Account }
$Regions            = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary            = Get-DoAWSDBInformation $Profiles $Regions
$Summary | Format-Table -AutoSize

### DynamoDB Tables

In [ ]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize

### RDS Instances

In [ ]:
$RDS        = Get-DoRDSDBSummary $Profiles $Regions 
$Snapshots  = Get-DoRDSSnapshotSummary $Profiles $Regions
$RDS | Format-Table -AutoSize

#### Snapshots

##### places-api

In [ ]:
$DB = 'places-api'
$Snapshots | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object $RDSSummaryColumns |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize